1️⃣ Use Hierarchical Forecasting   
  
What It Is   
	•	Hierarchical forecasting aggregates data at higher levels (e.g., brand level or total sales) to compensate for sparsity at lower levels (e.g., brand + family).   
	•	Predictions are made at the higher levels and disaggregated down to smaller groups (e.g., brand-family) using proportions.   
  
Steps to Implement   
	1.	Aggregate to Higher Levels:   
	•	Aggregate sparse brand-family data up to broader levels:   
	•	Brand level: brand_total_sales  
	•	Family level: family_total_sales  
	•	Overall total: global_total_sales  
	•	Use the aggregated time series to train the model.  
	2.	Forecast at Higher Levels:   
	•	Predict sales at the aggregated levels using richer historical data.  
	3.	Disaggregate Forecasts:  
	•	Split the higher-level forecasts back down to the brand-family level using historical proportions. For example:
$\text{brand\_family\_forecast} = \text{brand\_forecast} \times \frac{\text{brand\_family\_sales}}{\text{brand\_total\_sales}}$  
	4.	Smooth Proportions for Stability:  
	•	Use smoothed historical proportions (e.g., rolling averages) to prevent overfitting to noisy historical proportions.  



Steps in the Implementation  
	1.	Aggregate Sparse Data to Higher Levels (e.g., brand).    
	2.	Train a Forecasting Model at the Higher Level (e.g., using a simple statistical model or machine learning).    
	3.	Disaggregate Forecasts:    
	•	Use historical proportions to split higher-level forecasts back down to the brand + family level.  

In [1]:
import pandas as pd
import utils.utils_models as utils_models

import utils.utils_preprocessing as utils_preprocessing
import utils.utils_features as utils_features

In [2]:
df_sales = pd.read_parquet("data/sales.parquet")
df_products = pd.read_parquet("data/products.parquet")
df = df_sales.merge(df_products, on="product_id")

In [3]:
df = utils_preprocessing.set_types(df)

df_monthly = utils_preprocessing.aggregate_data(
    df, 
    frequency="ME"
    )
df_monthly = utils_preprocessing.fill_in_missing_dates(
    df_monthly, 
    group_col=["brand", "family"], 
    date_col="date", 
    freq="ME"
    )
df_monthly = utils_features.features_time_related(
    df_monthly, 
    frequency='ME'
    )
#df_monthly = df_monthly.set_index('date')

In [4]:
# Define the last 12 months for the test set (forecasting target)
forecast_horizon = 12

# Use the last month of your dataset as the reference to split
train_end_date = df_monthly['date'].max() - pd.DateOffset(months=forecast_horizon)

# Split into training and test sets
train_data = df_monthly[df_monthly['date'] <= train_end_date]
test_data = df_monthly[df_monthly['date'] > train_end_date]
X_train, X_test, y_train, y_test = utils_preprocessing.train_test_split(df_monthly, forecasting_horizon=12, target_col="total_revenue")

In [5]:
# Initialize a list to store rolling proportions for each month in training data
historical_proportions_train = []

for current_date in train_data['date'].unique():
    # Select training data up to the current month
    current_data_train = train_data[train_data['date'] <= current_date]
    
    # Aggregate revenue by brand and family for the current month
    current_revenue_train = current_data_train.groupby(['date', 'brand', 'family'])['total_revenue'].sum().reset_index()
    
    # Aggregate total revenue up to the current date (monthly)
    total_revenue_up_to_date_train = current_data_train.groupby(['date'])['total_revenue'].sum().reset_index()
    
    # Merge to calculate proportions (revenue share of each brand/family per month)
    current_revenue_train = current_revenue_train.merge(total_revenue_up_to_date_train, on='date', suffixes=('', '_agg'))
    
    # Calculate proportions: revenue of brand/family over total revenue for that month
    current_revenue_train['proportion'] = current_revenue_train['total_revenue'] / current_revenue_train['total_revenue_agg']
    
    # Store the proportions data
    historical_proportions_train.append(current_revenue_train[['date', 'brand', 'family', 'proportion']])

# Combine all the monthly proportions into a single DataFrame
historical_proportions_train_df = pd.concat(historical_proportions_train)
historical_proportions_train_df['month'] = historical_proportions_train_df['date'].dt.month
historical_proportions_train_df = historical_proportions_train_df.drop(columns='date')
historical_proportions_train_df = historical_proportions_train_df.groupby(['brand', 'family', 'month'], as_index=False).agg({'proportion': 'mean'})

In [6]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Forecasting the total revenue using SARIMA model
sarima_model = SARIMAX(train_data.groupby('date')['total_revenue'].sum(), order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
sarima_result = sarima_model.fit()

# Forecast total revenue for the last 12 months
forecast_total_revenue = sarima_result.get_forecast(steps=forecast_horizon).predicted_mean
forecast_total_revenue = forecast_total_revenue.reset_index()
forecast_total_revenue.columns = ['date', 'forecast_total_revenue']
forecast_total_revenue['month'] = forecast_total_revenue['date'].dt.month
#forecast_total_revenue = forecast_total_revenue[['month', 'forecast_total_revenue']]

/Users/matteogiaretti/Python/use_case_deporvillage/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)
/Users/matteogiaretti/Python/use_case_deporvillage/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)
/Users/matteogiaretti/Python/use_case_deporvillage/.venv/lib/python3.13/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


In [7]:
forecast_allocation = X_test.reset_index()[['date', 'brand', 'family']].merge(forecast_total_revenue, on='date', how='left')

In [8]:
forecast_allocation = forecast_allocation.merge(historical_proportions_train_df, on=['brand', 'family', 'month'], how='left')

In [9]:
#forecast_allocation = pd.merge(historical_proportions_train_df, forecast_total_revenue, on='month', how='left')

# Calculate forecasted revenue for each brand + family
forecast_allocation['forecast_revenue'] = (
    forecast_allocation['forecast_total_revenue'] * forecast_allocation['proportion']
)

# Final output: forecasted revenue at brand + family level
forecast_allocation = forecast_allocation[['date', 'brand', 'family', 'forecast_revenue']]

In [10]:
forecast_allocation

,date,brand,family,forecast_revenue
0,2023-10-31,adidas,Headwear Urban,60.981145
1,2023-10-31,adidas,Zapatillas Running,19622.427604
2,2023-10-31,adidas,Polos Urban,143.257943
3,2023-10-31,Columbia,Pantalones Urban,NaN
4,2023-10-31,adidas,Mallas y pantalones Training,1248.555961
...,...,...,...,...
819,2024-09-30,adidas,Pantalones Urban,1429.126322
820,2024-09-30,Columbia,Headwear Outdoor,362.273874
821,2024-09-30,adidas,Pantalones Esquí,347.503691
822,2024-09-30,adidas,Tops y Sujetadores deportivos Training,551.900316


In [12]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
forecast_allocation = forecast_allocation.fillna(0)

# Assuming 'forecast_allocation' contains the forecasted revenues for the test period
# Merge forecasted values with actual values in the test data
actual_vs_forecast = pd.merge(test_data[['date', 'brand', 'family', 'total_revenue']], forecast_allocation, on=['date', 'brand', 'family'], how='left')

# Evaluate using MAE and RMSE
mae = mean_absolute_error(actual_vs_forecast['total_revenue'], actual_vs_forecast['forecast_revenue'])
rmse = mean_squared_error(actual_vs_forecast['total_revenue'], actual_vs_forecast['forecast_revenue'], squared=False)

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

TypeError: got an unexpected keyword argument 'squared'

In [13]:
actual_vs_forecast

,date,brand,family,total_revenue,forecast_revenue
0,2023-10-31,Columbia,Bañadores Natación,99.949997,110.397347
1,2023-11-30,Columbia,Bañadores Natación,99.949997,90.428565
2,2023-12-31,Columbia,Bañadores Natación,99.949997,91.267492
3,2024-01-31,Columbia,Bañadores Natación,99.949997,99.368503
4,2024-02-29,Columbia,Bañadores Natación,99.949997,114.270362
...,...,...,...,...,...
819,2024-03-31,adidas,Zuecos Urban,99.949997,115.963923
820,2024-04-30,adidas,Zuecos Urban,99.949997,175.494766
821,2024-05-31,adidas,Zuecos Urban,199.899994,168.540355
822,2024-06-30,adidas,Zuecos Urban,99.949997,181.505176


In [14]:
import numpy as np

In [ ]:
round(np.sqrt(mean_squared_error(y_test, y_pred)), 3)